In [1]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture

In [2]:
df = pd.read_csv('../data/fashion/fashion-mnist_train.csv')

In [3]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
y = df['label']
del df['label']

In [6]:
%%time
gmm = GaussianMixture(n_components=10, covariance_type='diag').fit(df)

CPU times: user 1min 18s, sys: 16.8 s, total: 1min 35s
Wall time: 49.4 s


In [7]:
gmm.means_[0]

array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   1.54834015e-03,
         4.36631923e-02,   6.90559708e-02,   3.40015497e-01,
         1.76015312e+00,   4.72522448e+00,   8.19774162e+00,
         1.03842482e+01,   1.18358218e+01,   1.21702633e+01,
         1.22710034e+01,   1.18540996e+01,   1.15104502e+01,
         1.14530738e+01,   1.08417290e+01,   9.28260888e+00,
         5.51054260e+00,   3.23107623e+00,   1.38421610e+00,
         3.01306994e-01,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         8.67070485e-03,   1.05287130e-01,   3.04713342e-01,
         1.98899779e+00,   7.78823287e+00,   1.80729593e+01,
         2.81052467e+01,   3.51818039e+01,   3.91479253e+01,
         4.08069570e+01,   4.11891393e+01,   4.10530991e+01,
         4.22857563e+01,

In [9]:
gmm.weights_

array([ 0.05382108,  0.02853285,  0.0355013 ,  0.19483333,  0.13738093,
        0.1826    ,  0.17520001,  0.08996667,  0.07461387,  0.02754997])

In [10]:
gmm.covariances_

array([[  1.00000000e-06,   1.00000000e-06,   1.00000000e-06, ...,
          1.76492625e+03,   7.91402237e+02,   2.13915893e+01],
       [  2.95577273e-01,   4.10937487e-01,   6.94963533e-01, ...,
          8.77590633e+02,   2.75102085e+02,   7.77109141e+00],
       [  1.45172117e-02,   1.72250793e+00,   4.06840368e+01, ...,
          2.04501739e+03,   4.05806244e+02,   3.09952685e+01],
       ..., 
       [  1.00000000e-06,   1.00000000e-06,   1.00000000e-06, ...,
          1.00000000e-06,   1.00000000e-06,   1.00000000e-06],
       [  1.00000000e-06,   1.00000000e-06,   2.14295658e-01, ...,
          3.18809952e-02,   1.00000000e-06,   1.00000000e-06],
       [  1.00000000e-06,   1.00000000e-06,   1.00000000e-06, ...,
          1.70591120e+03,   6.54560478e+02,   7.30949011e+01]])

In [8]:
%%time
gmm2 = GaussianMixture(n_components=10, covariance_type='full').fit(df)

CPU times: user 54min 46s, sys: 5min 28s, total: 1h 15s
Wall time: 33min 59s


In [18]:
gmm2.means_[0]

array([  1.48669453e-03,   1.42297905e-02,   7.47594965e-02,
         5.13759160e-01,   1.24605848e+00,   1.92907245e+00,
         3.06259075e+00,   7.11829413e+00,   1.58165096e+01,
         3.05525545e+01,   5.07010520e+01,   7.59895767e+01,
         8.81925975e+01,   7.80554665e+01,   7.72660318e+01,
         8.73075629e+01,   8.38470873e+01,   5.58844258e+01,
         3.40023503e+01,   1.92252763e+01,   8.82756714e+00,
         3.95258104e+00,   2.37616297e+00,   1.74877755e+00,
         1.13328585e+00,   4.69158317e-01,   1.15112641e-01,
         3.18577400e-03,   3.78045181e-02,   1.11289705e-01,
         3.76983256e-01,   2.44943151e+00,   4.76803391e+00,
         1.02571305e+01,   3.08244882e+01,   5.85276982e+01,
         8.32940947e+01,   1.06131490e+02,   1.32123434e+02,
         1.70992643e+02,   1.82781700e+02,   1.75773658e+02,
         1.74608856e+02,   1.82813491e+02,   1.75278616e+02,
         1.37499070e+02,   1.10361511e+02,   8.84823685e+01,
         6.45364083e+01,

In [11]:
gmm.weights_

array([ 0.05382108,  0.02853285,  0.0355013 ,  0.19483333,  0.13738093,
        0.1826    ,  0.17520001,  0.08996667,  0.07461387,  0.02754997])

In [12]:
gmm.covariances_

array([[  1.00000000e-06,   1.00000000e-06,   1.00000000e-06, ...,
          1.76492625e+03,   7.91402237e+02,   2.13915893e+01],
       [  2.95577273e-01,   4.10937487e-01,   6.94963533e-01, ...,
          8.77590633e+02,   2.75102085e+02,   7.77109141e+00],
       [  1.45172117e-02,   1.72250793e+00,   4.06840368e+01, ...,
          2.04501739e+03,   4.05806244e+02,   3.09952685e+01],
       ..., 
       [  1.00000000e-06,   1.00000000e-06,   1.00000000e-06, ...,
          1.00000000e-06,   1.00000000e-06,   1.00000000e-06],
       [  1.00000000e-06,   1.00000000e-06,   2.14295658e-01, ...,
          3.18809952e-02,   1.00000000e-06,   1.00000000e-06],
       [  1.00000000e-06,   1.00000000e-06,   1.00000000e-06, ...,
          1.70591120e+03,   6.54560478e+02,   7.30949011e+01]])